<a href="https://colab.research.google.com/github/Ijeoma-Charity/ai-ml-experiments/blob/main/Input_Target_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import files
uploaded = files.upload()

Saving the_verdict.docx to the_verdict.docx


In [24]:
pip install python-docx pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00


In [25]:
from docx import Document

In [26]:
doc = Document("the_verdict.docx")
full_text = '/n'.join(p.text for p in doc.paragraphs)

In [27]:
with open("the_verdict.txt", "w", encoding = "utf-8") as f:
  f.write(full_text)

In [28]:
with open("the_verdict.txt", "r", encoding = "utf-8") as f:
  raw_text = f.read()

installing tiktoken

In [31]:
! pip3 install tiktoken

In [32]:
import tiktoken

Initializing the tokenizer

In [33]:
tokenizer = tiktoken.get_encoding("gpt2")

In [34]:
enc_text = tokenizer.encode(raw_text)
enc_sample = enc_text[50:]
print(len(enc_sample))

5264


Introduction into the input-target pair concept

In [36]:
#Understanding the basics of the input-target pair
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [326, 11, 287, 262]
y:      [11, 287, 262, 6001]


In [37]:
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

print(context, "---->", desired)
print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[326, 11, 287, 262] ----> 6001
 that, in the ---->  height


Creating a Data Loader

In [19]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

  #tokenize the entire text
    token_ids = tokenizer.encode(text,allowed_special = {"<|endoftext|>"})

  #implementing the sliding window to chunk the book into overlapping sequence of max_length
    for i in range(0, len(token_ids)-max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx],self.target_ids[idx]

Creating a data loader

In [20]:
def create_dataloader_v1(text,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):

  #initialize tokenizer
  tokenizer = tiktoken.get_encoding("gpt2")

  #creating a dataset
  dataset = GPTDatasetV1(text, tokenizer, max_length,stride)

  #creating a dataloader
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )

  return dataloader

Testing the data loader

In [16]:
with open("the_verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [17]:
import torch
print("Pytorch version: ", torch.__version__)

Pytorch version:  2.9.0+cpu


In [18]:
dataloader = create_dataloader_v1(raw_text,batch_size=1,stride=1,max_length=4,shuffle=False)

In [19]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  464,  4643, 11600,  1717]]), tensor([[ 4643, 11600,  1717,   342]])]


In [20]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 4643, 11600,  1717,   342]]), tensor([[11600,  1717,   342,   854]])]


Experimenting the effect of batch_size

In [21]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)

In [22]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("inputs:\n", inputs)
print("\nTargets:\n", targets)

inputs:
 tensor([[  464,  4643, 11600,  1717],
        [  342,   854, 41328, 40417],
        [ 1475,  9213,   422, 11145],
        [  271,  1668,   319,  3426],
        [ 1596,    11, 32190,   352],
        [  314,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026]])

Targets:
 tensor([[ 4643, 11600,  1717,   342],
        [  854, 41328, 40417,  1475],
        [ 9213,   422, 11145,   271],
        [ 1668,   319,  3426,  1596],
        [   11, 32190,   352,   314],
        [  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632]])


**VECTOR EMBEDDINGS**

In [3]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 89.7 MB/s eta 0:00:00


In [4]:
#Import trained model
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
print(model["computer"])
print(model["cat"].shape)

[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  5.02929

In [6]:
print(model.most_similar("king"))

[('kings', 0.7138045430183411), ('queen', 0.6510956883430481), ('monarch', 0.6413194537162781), ('crown_prince', 0.6204220056533813), ('prince', 0.6159993410110474), ('sultan', 0.5864824056625366), ('ruler', 0.5797567367553711), ('princes', 0.5646552443504333), ('Prince_Paras', 0.5432944297790527), ('throne', 0.5422105193138123)]


In [7]:
print(model.most_similar(positive=["king", "woman"], negative=["man"], topn=10))

[('queen', 0.7118193507194519), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321839332581), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.518113374710083), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411403656006)]


CHECKING FOR SIMILARITIES BETWEEN WORDS

In [8]:
import numpy as np
#words to compare
word1 = "man"
word2 = "woman"

word3 = "semiconductor"
word4 = "earthworm"

word5 = "nephew"
word6 = "niece"

In [9]:
#calculate the vector difference
vector_difference1 = model[word1] - model[word2]
vector_difference2 = model[word3] - model[word4]
vector_difference3 = model[word5] - model[word6]

In [10]:
magnitude_of_difference1 = np.linalg.norm(vector_difference1)
print(magnitude_of_difference1)
magnitude_of_difference2 = np.linalg.norm(vector_difference2)
print(magnitude_of_difference2)
magnitude_of_difference3 = np.linalg.norm(vector_difference3)
print(magnitude_of_difference3)

print("The magnitude of the difference between '{}' and '{}' is {:.2f}".format(word1,word2,magnitude_of_difference1))
print("The magnitude of the difference between '{}' and '{}' is {:.2f}".format(word3,word4,magnitude_of_difference2))
print("The magnitude of the difference between '{}' and '{}' is {:.2f}".format(word4,word5,magnitude_of_difference3))

1.727951
5.6670427
1.9557794
The magnitude of the difference between 'man' and 'woman' is 1.73
The magnitude of the difference between 'semiconductor' and 'earthworm' is 5.67
The magnitude of the difference between 'earthworm' and 'nephew' is 1.96


CREATING TOKEN EMBEDDINGS

In [11]:
import torch

In [12]:
input_ids = torch.tensor([2,3,5,1])

In [13]:
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [14]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [15]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [16]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


IMPLEMENTING POSITIONAL EMBEDDING

In [17]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [41]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
    )
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token_IDs:\n", inputs)
print("\nInuts shape:\n", inputs.shape)

Token_IDs:
 tensor([[  464,  4643, 11600,  1717],
        [  342,   854, 41328, 40417],
        [ 1475,  9213,   422, 11145],
        [  271,  1668,   319,  3426],
        [ 1596,    11, 32190,   352],
        [  314,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026]])

Inuts shape:
 torch.Size([8, 4])


In [43]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [45]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [47]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
